In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

df = pd.DataFrame(data=None, index=None, columns=("School", "BAGradRate4yr", "IPEDS_ID"))

for j in collegeslinks:

    page = requests.get(j)
    IPEDS_ID = str(j)[-6:]

    soup = BeautifulSoup(page.content, 'html.parser')

    name_box = soup.find('div', attrs = {'id':'retgrad'}) 

    if name_box is None:
        continue

    page_title = soup.title.text

    page_body = soup.body

    table = soup.find('div', attrs = {'id':'retgrad'}) 

    tags = []
    collection = table.findAll("img")
    for img in collection:
        if 'alt' in img.attrs:
            tags.append(img.attrs['alt'])

    matches = [bool(re.search("Bachelor's degree rate", str_)) for str_ in tags]

    res = [i for i, val in enumerate(matches) if val]
    res = str(res)
    res = res.replace("[", "")
    res = res.replace("]", "")

    c = int(res) if res else None

    b = (tags[c]) if c else None

    b = re.sub(".*Bachelor's degree rate:", "", b) if b else None
    b = re.sub("8-year: \\d{1,2}\\%|6-year: \\d{1,2}\\%", "", b) if b else None

    page_title = re.sub(".*College Navigator -", "", page_title) if page_title else None

    df = df.append({'School':page_title, "IPEDS_ID":IPEDS_ID, "BAGradRate4yr":b}, ignore_index=True)

    df['School'] = df['School'].str.extract(r'([^\r\n]+)', expand=False)
        
    df['BAGradRate4yr_fall_2013_cohort'] = df['BAGradRate4yr'].str.extract(r'(\d?\d\%+)', expand=False)
    df['BAGradRate4yr_fall_2015_cohort'] = df['BAGradRate4yr'].str.findall(r'(\d?\d\%)').str[1]


df = df.drop("BAGradRate4yr", axis=1)
df.to_csv('df.csv')